# U-Net model
Network architecture called "U-Net". The name of this network architecture comes from it's U-like shape.

U-nets are commonly used for image segmentation, this architecture features a series of down-convolutions connected by max-pooling operations, followed by a series of up-convolutions connected by upsampling and concatenation operations. Each of the down-convolutions is also connected directly to the concatenation operations in the upsampling portion of the network. For more detail on the U-Net architecture, have a look at the original U-Net paper by Ronneberger et al. 2015.


In [1]:
# Import the elements we'll need to build your U-Net
import keras
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D
from keras.optimizers import Adam
from keras.layers.merge import concatenate
# Set the image shape to have the channels in the first dimension
K.set_image_data_format("channels_first")

Using TensorFlow backend.


## The "depth" of  U-Net
The "depth" of our U-Net is equal to the number of down-convolutions we will use. U-Net depth of 2, meaning we'll have 2 down-convolutions in your network.

## Input layer and its "depth"
we will be doing 3D image segmentation, which is to say that, in addition to "height" and "width", our input layer will also have a "length".

The shape of the input layer is (num_channels, height, width, length), where num_channels you can think of like color channels in an image, height, width and length are just the size of the input.

here, the values will be:

num_channels: 4
height: 160
width: 160
length: 16

In [2]:
# input layer tensor of the shape you'll use in the assignment
input_layer = Input(shape=(4, 160, 160, 16))
input_layer

<tf.Tensor 'input_1:0' shape=(None, 4, 160, 160, 16) dtype=float32>

Notice that the tensor shape has a '?' as the very first dimension. This will be the batch size. So the dimensions of the tensor are: (batch_size, num_channels, height, width, length)

## Contracting (downward) path
Here we'll start by constructing the downward path in your network (the left side of the U-Net). The (height,width, length) of the input gets smaller as you move down this path, and the number of channels increases.

Depth 0
By "depth 0" here, we're referring to the depth of the first down-convolution in the U-net.

The number of filters is specified for each depth and for each layer within that depth.

The formula to use for calculating the number of filters is:
filtersi=32×(2i)
Where i is the current depth.

So at depth i=0:
filters0=32×(20)=32
## Layer 0
There are two convolutional layers for each depth


In [3]:
# Conv3D tensor with 32 filters
down_depth_0_layer_0 = Conv3D(filters=32, 
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                              )(input_layer)
down_depth_0_layer_0

<tf.Tensor 'conv3d_1/add:0' shape=(None, 32, 160, 160, 16) dtype=float32>

Notice that with 32 filters, the result you get above is a tensor with 32 channels.


In [4]:
#a relu activation to layer 0 of depth 0
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)
down_depth_0_layer_0

<tf.Tensor 'activation_1/Relu:0' shape=(None, 32, 160, 160, 16) dtype=float32>

## Depth 0, Layer 1
For layer 1 of depth 0, the formula for calculating the number of filters is:

filters[i]=32×(2pow(i))×2
 
Where  i  is the current depth.

Notice that the ' × 2 ' at the end of this expression isn't there for layer 0.
So at depth  i=0  for layer 1:
filters[0]=32×(20)×2=64

In [5]:
# Conv3D layer with 64 filters and add relu activation
down_depth_0_layer_1 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_0)
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)
down_depth_0_layer_1

<tf.Tensor 'activation_2/Relu:0' shape=(None, 64, 160, 160, 16) dtype=float32>

## Max pooling
Within the U-Net architecture, there is a max pooling operation after each of the down-convolutions (not including the last down-convolution at the bottom of the U). In general, this means we'll add max pooling after each down-convolution up to (but not including) the depth - 1 down-convolution (since we started counting at 0).

Here:

The overall depth of the U-Net you're constructing is 2
So the bottom of  U is at a depth index of:  2−1=1 .
So far we've only defined the  depth=0  down-convolutions, so the next thing to do is add max pooling


In [6]:
#  max pooling layer
down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)
down_depth_0_layer_pool

<tf.Tensor 'max_pooling3d_1/MaxPool3D:0' shape=(None, 64, 80, 80, 8) dtype=float32>

## Depth 1, Layer 0
At depth 1, layer 0, the formula for calculating the number of filters is:
filtersi=32×(2i)
 
Where  i  is the current depth.

So at depth  i=1 :
filters[1]=32×(21)=64
 

In [7]:
# Conv3D layer to your network with relu activation
down_depth_1_layer_0 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_pool)
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)
down_depth_1_layer_0

<tf.Tensor 'activation_3/Relu:0' shape=(None, 64, 80, 80, 8) dtype=float32>

## Depth 1, Layer 1
For layer 1 of depth 1 the formula we'll use for number of filters is:
filters[i]=32×(2i)×2
 
Where  i  is the current depth.

Notice that the ' ×2 ' at the end of this expression isn't there for layer 0.
So at depth  i=1 :

filters[0]=32×(21)×2=128
 

In [8]:
#  another Conv3D with 128 filters to your network.
down_depth_1_layer_1 = Conv3D(filters=128, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)
down_depth_1_layer_1

<tf.Tensor 'activation_4/Relu:0' shape=(None, 128, 80, 80, 8) dtype=float32>

## No max pooling at depth 1 (the bottom of the U)
When we get to the "bottom" of the U-net, we don't need to apply max pooling after the convolutions.

## Expanding (upward) Path
Now we'll work on the expanding path of the U-Net, (going up on the right side ). The image's (height, width, length) all get larger in the expanding path.

## Depth 0, Up sampling layer 0
We'll use a pool size of (2,2,2) for upsampling.


In [9]:
# an upsampling operation to your network
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)
up_depth_0_layer_0

<tf.Tensor 'up_sampling3d_1/concat_2:0' shape=(None, 128, 160, 160, 16) dtype=float32>

## Concatenate upsampled depth 0 with downsampled depth 0
Now we'll apply a concatenation operation using the layers that are both at the same depth of 0.

up_depth_0_layer_0: shape is (?, 128, 160, 160, 16)
depth_0_layer_1: shape is (?, 64, 160, 160, 16)


If they're the same, then they can be concatenated along axis 1 (the channel axis).
The (height, width, length) is (160, 160, 16) for both.


In [10]:
# Print the shape of layers to concatenate
print(up_depth_0_layer_0)
print()
print(down_depth_0_layer_1)

Tensor("up_sampling3d_1/concat_2:0", shape=(None, 128, 160, 160, 16), dtype=float32)

Tensor("activation_2/Relu:0", shape=(None, 64, 160, 160, 16), dtype=float32)


In [11]:
# concatenation along axis 1
up_depth_1_concat = concatenate([up_depth_0_layer_0,
                                 down_depth_0_layer_1],
                                axis=1)
up_depth_1_concat

<tf.Tensor 'concatenate_1/concat:0' shape=(None, 192, 160, 160, 16) dtype=float32>

Notice that the upsampling layer had 128 channels, and the down-convolution layer had 64 channels so that when concatenated, the result has 128 + 64 = 192 channels.

## Up-convolution layer 1
The number of filters for this layer will be set to the number of channels in the down-convolution's layer 1 at the same depth of 0 (down_depth_0_layer_1).


In [12]:
down_depth_0_layer_1

<tf.Tensor 'activation_2/Relu:0' shape=(None, 64, 160, 160, 16) dtype=float32>

In [13]:
print(f"number of filters: {down_depth_0_layer_1._keras_shape[1]}")

number of filters: 64


In [14]:
# Conv3D up-convolution with 64 filters to  network
up_depth_1_layer_1 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_concat)
up_depth_1_layer_1 = Activation('relu')(up_depth_1_layer_1)
up_depth_1_layer_1

<tf.Tensor 'activation_5/Relu:0' shape=(None, 64, 160, 160, 16) dtype=float32>

## Up-convolution depth 0, layer 2
At layer 2 of depth 0 in the up-convolution the next step will be to add another up-convolution. The number of filters we'll want to use for this next up-convolution will need to be equal to the number of filters in the down-convolution depth 0 layer 1.



In [15]:
print(down_depth_0_layer_1)
print(f"number of filters: {down_depth_0_layer_1._keras_shape[1]}")

Tensor("activation_2/Relu:0", shape=(None, 64, 160, 160, 16), dtype=float32)
number of filters: 64


As you can see, the number of channels / filters in down_depth_0_layer_1 is 64.


In [16]:
# Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_2 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_layer_1)
up_depth_1_layer_2 = Activation('relu')(up_depth_1_layer_2)
up_depth_1_layer_2

<tf.Tensor 'activation_6/Relu:0' shape=(None, 64, 160, 160, 16) dtype=float32>

## Final Convolution
For the final convolution, we will set the number of filters to be equal to the number of classes in our input data.

we will be using data with 3 classes, namely:

1: edema

2: non-enhancing tumor

3: enhancing tumor



In [17]:
# final Conv3D with 3 filters to your network.
final_conv = Conv3D(filters=3, #3 categories 
                    kernel_size=(1,1,1),
                    padding='valid',
                    strides=(1,1,1)
                    )(up_depth_1_layer_2)
final_conv

<tf.Tensor 'conv3d_7/add:0' shape=(None, 3, 160, 160, 16) dtype=float32>

### Activation for final convolution


In [18]:
# sigmoid activation to your final convolution.
final_activation = Activation('sigmoid')(final_conv)
final_activation

<tf.Tensor 'activation_7/Sigmoid:0' shape=(None, 3, 160, 160, 16) dtype=float32>

### Create and compile the model

In [19]:
# Define and compile the model
model = Model(inputs=input_layer, outputs=final_activation)
model.compile(optimizer=Adam(lr=0.00001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy']
             )

In [20]:
# Print out a summary of the model  created
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 160, 160,  0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 160, 160, 3488        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 160, 160, 0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 64, 160, 160, 55360       activation_1[0][0]               
____________________________________________________________________________________________

Now, we will use these techniques in our project in order to segemnet brain tumors